In [47]:
""" Okay, why is there a memory leak with my lpips stuff?"""
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import checkpoints
from torch.autograd import Variable
import lpips.dist_model as dm


In [46]:
""" Forward breaks lpips?
I have definitely witnessed a forward pass of my PerceptualXentropy loss function that 
causes a memout error with cuda. The goal of this block is to try and replicate that.
"""

def forward_breaks_lpips_1():
    # General strat: get cifar loader and run arbitrary distance comparisons until we break 
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, use_gpu=True)
    
    first_batch, _ = next(iter(val_loader))    
    lpips_regularizer = lf.LpipsRegularization(Variable(first_batch.cuda()), use_gpu=True)
    for i in xrange(10): # 10 epochs 
        print "EPoch %s" % i
        for i, (examples, labels) in enumerate(val_loader):
            if examples.shape[0] != first_batch.shape[0]:
                continue
            examples = Variable(examples.cuda(), requires_grad=True)
            # print examples.shape
            fix_im = lpips_regularizer.fix_im
            torch.sum(lpips_regularizer.dist_model.forward_var(2*examples -1 , 2*fix_im-1))
            out = torch.sum(lpips_regularizer.forward(examples))
            out.backward()
            
            #_ = lpips_regularizer.forward(examples)
            


forward_breaks_lpips_1()



Files already downloaded and verified
EPoch 0
EPoch 1
EPoch 2
EPoch 3
EPoch 4
EPoch 5
EPoch 6
EPoch 7
EPoch 8
EPoch 9


In [30]:
# understand dist model 

def understand_dist_model():
    dist_model = dm.DistModel()
    dist_model.initialize(model='net-lin', net='alex', use_gpu=True)
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, use_gpu=True)    
    first_batch, _ = next(iter(val_loader))    
    first_batch = first_batch.cuda()
    xform = lambda img: img * 2 - 1

    first_stack = xform(first_batch[:5])
    second_stack = xform(first_batch[5:10])
    
    make_var = lambda tensor: Variable(tensor, requires_grad=True)
    first_var = make_var(first_stack)
    second_var = make_var(second_stack)
    
    out = torch.sum(dist_model.forward_var(first_var, second_var))
    
    print out 
    out.backward()
    print first_var.grad
    
    
    
understand_dist_model()

    

Files already downloaded and verified
Variable containing:
 0.5658
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
(0 ,0 ,.,.) = 
  7.2645e-05  1.5882e-04  5.7358e-05  ...  -1.0450e-04 -2.2553e-05 -8.1600e-06
  1.1721e-04  1.3188e-04 -7.3912e-06  ...  -2.2492e-04 -1.9046e-05  1.8680e-05
 -4.6280e-05  1.4303e-04  7.9768e-05  ...  -3.7147e-05  1.4194e-04  1.2253e-04
                 ...                   ⋱                   ...                
 -8.0557e-05 -1.6988e-04 -3.8638e-04  ...  -2.0251e-04 -2.0716e-04 -5.7161e-05
  1.0974e-04  1.7803e-04  2.7093e-04  ...  -1.9485e-04 -1.6650e-04 -1.2187e-05
 -1.2900e-04 -1.6780e-04 -1.5961e-04  ...  -1.4606e-04 -1.0146e-04  1.8680e-05

(0 ,1 ,.,.) = 
 -2.5180e-05  1.1779e-04  2.5061e-04  ...  -1.7541e-04 -8.0358e-05 -3.9741e-05
 -3.4342e-05  5.6369e-05  1.8414e-04  ...  -3.1962e-04 -8.9423e-05 -2.5685e-05
 -2.9653e-04 -1.1740e-05  1.8767e-04  ...  -2.7568e-04 -5.2932e-05 -6.0643e-06
                 ...                   ⋱       